We will now implement SwinUNETR, a transformer-based segmentation model that has shown superior performance in medical image segmentation tasks, especially in 3D datasets.

📌 Why SwinUNETR?
✅ Uses Swin Transformer as an encoder for long-range spatial dependencies
✅ Outperforms CNN-based models like U-Net on small datasets
✅ Works well for multi-scale feature extraction in 3D medical images

🔷 Workflow Overview
Similar to nnUNet, the workflow includes:

Preprocessing (Cropping, Normalization, Augmentation)
Manual Annotation (3D Slicer)
Human-in-the-Loop Approach
Model Training (SwinUNETR)
Evaluation & Deployment

This is the step-by-step code that you can execute in a Jupyter Notebook.

🔷 Step 1: Preprocessing
Since you've already cropped the laryngeal region and converted .nrrd → .nii.gz, we continue with data normalization and augmentation.

🔹 1.1 Install MONAI
SwinUNETR is implemented in MONAI, which is built on PyTorch.

In [ ]:
!pip install pynrrd 
!pip installnibabel 
!pip installnumpy
!pip install monai
!pip install torch
!pip install torchvision

In [ ]:
#Convert .nrrd to .nii.gz
import nrrd
import nibabel as nib
import numpy as np
import os

# Define input and output directories
input_folder = "/path/to/nrrd/"
output_folder = "/path/to/nifti/"

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".nrrd"):
        file_path = os.path.join(input_folder, file)
        data, header = nrrd.read(file_path)  # Read NRRD file
        
        # Convert to NIfTI format
        nifti_img = nib.Nifti1Image(data, affine=np.eye(4))
        
        # Save the converted file
        output_path = os.path.join(output_folder, file.replace(".nrrd", ".nii.gz"))
        nib.save(nifti_img, output_path)
        print(f"Converted: {file} → {output_path}")


✅ Output: All nrrd files are converted to .nii.gz.

Crop Laryngeal Region

In [ ]:
!pip install SimpleITK

In [ ]:
#Cropping Code
import SimpleITK as sitk
import os

input_folder = "/path/to/nifti/"
output_folder = "/path/to/cropped_nifti/"
os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".nii.gz"):
        img_path = os.path.join(input_folder, file)
        
        # Read the image
        img = sitk.ReadImage(img_path)
        array = sitk.GetArrayFromImage(img)

        # Define crop range (adjust based on dataset)
        crop_slices = (50, 150)  # Adjust Z-axis range
        cropped_array = array[crop_slices[0]:crop_slices[1], :, :]
        
        # Convert back to NIfTI
        cropped_img = sitk.GetImageFromArray(cropped_array)
        cropped_img.SetSpacing(img.GetSpacing())
        cropped_img.SetDirection(img.GetDirection())
        cropped_img.SetOrigin(img.GetOrigin())

        # Save cropped image
        output_path = os.path.join(output_folder, file)
        sitk.WriteImage(cropped_img, output_path)
        print(f"Cropped and saved: {file}")


In [ ]:
#Normalize Images

import nibabel as nib
import numpy as np
import os

input_folder = "/path/to/cropped_nifti/"
output_folder = "/path/to/normalized_nifti/"
os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".nii.gz"):
        img_path = os.path.join(input_folder, file)
        
        # Load image
        img = nib.load(img_path)
        img_data = img.get_fdata()
        
        # Normalize: Convert HU range (-1000 to 1000) → [0,1]
        img_data = np.clip(img_data, -1000, 1000)
        img_data = (img_data + 1000) / 2000  # Normalize to [0,1]
        
        # Save normalized image
        normalized_img = nib.Nifti1Image(img_data, img.affine)
        output_path = os.path.join(output_folder, file)
        nib.save(normalized_img, output_path)
        print(f"Normalized: {file}")


✅ Output: Normalized CT images scaled to [0,1].

🔷 Step 2: Annotation Using 3D Slicer
✅ Manually segment 50-100 cases in 3D Slicer
✅ Export segmentations as .nii.gz files

🔹 Steps
Open 3D Slicer and load the cropped, normalized CT scans.
Use Segment Editor to manually label the thyroid cartilage.
Save as .nii.gz in /path/to/labels/.

🔷 Step 3: Human-in-the-Loop (HITL) Approach
We train an initial SwinUNETR model on manually labeled images and use it to generate pseudo-labels.

🔹 Steps
Train initial SwinUNETR on manually annotated 50-100 cases.
Use the trained model to predict segmentations on unlabeled data.
Refine pseudo-labels manually using 3D Slicer.
Retrain SwinUNETR on the refined dataset.

🔷 Step 4: SwinUNETR Model Training

In [ ]:
#Import Required Libraries
import os
import torch
import monai
from monai.networks.nets import SwinUNETR
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, Spacingd,
    ScaleIntensityRanged, CropForegroundd, RandCropByPosNegLabeld,
    ToTensord
)
from monai.data import Dataset, DataLoader
from monai.losses import DiceLoss
from monai.optimizers.lr_scheduler import WarmupCosineSchedule
from monai.metrics import DiceMetric
import matplotlib.pyplot as plt


In [ ]:
#4.2 Define Dataset Paths
data_dir = "/path/to/normalized_nifti/"
label_dir = "/path/to/labels/"

train_images = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
train_labels = [os.path.join(label_dir, f) for f in os.listdir(label_dir)]

train_files = [{"image": img, "label": lbl} for img, lbl in zip(train_images, train_labels)]


In [ ]:
#4.3 Define Data Augmentation & Preprocessing
transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
    ScaleIntensityRanged(keys=["image"], a_min=0, a_max=1, b_min=0, b_max=1, clip=True),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    RandCropByPosNegLabeld(keys=["image", "label"], spatial_size=(96,96,96), pos=1, neg=1, num_samples=4),
    ToTensord(keys=["image", "label"])
])


In [ ]:
#4.4 Create DataLoader
train_dataset = Dataset(data=train_files, transform=transforms)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)


In [ ]:
#4.5 Define SwinUNETR Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SwinUNETR(
    img_size=(96,96,96),
    in_channels=1,
    out_channels=2,
    feature_size=48,
    use_checkpoint=True
).to(device)

loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
lr_scheduler = WarmupCosineSchedule(optimizer, warmup_iters=10, max_iters=100)
dice_metric = DiceMetric(include_background=False, reduction="mean")


In [ ]:
#4.6 Train SwinUNETR Model
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch_data in train_loader:
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")


✅ Model is trained for 50 epochs.

🔷 Step 5: Model Evaluation & Deployment

In [ ]:
#5.1 Evaluate Model on Test Data
model.eval()
with torch.no_grad():
    for batch_data in train_loader:
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)
        outputs = model(inputs)
        dice_score = dice_metric(outputs, labels)
        print(f"Dice Score: {dice_score.mean().item():.4f}")


✅ Measures model accuracy using Dice score.

In [ ]:
#5.2 Inference on New Images
test_image = "/path/to/new_ct.nii.gz"
input_data = transforms({"image": test_image})
input_tensor = input_data["image"].unsqueeze(0).to(device)

model.eval()
output = model(input_tensor)
predicted_mask = torch.argmax(output, dim=1).cpu().numpy()


✅ Generates automatic segmentations on new CT scans.